Following guide from [lifelib](https://lifelib.io/libraries/notebooks/basiclife/generate_model_points.html)

# [Generating model points for ASL](https://lifelib.io/libraries/notebooks/basiclife/generate_model_points_ASL.html)

In [3]:
import numpy as np
from numpy.random import default_rng  # Requires NumPy 1.17 or newer
import pandas as pd
import uuid

rng = default_rng(12345)

# Number of Model Points
point_size = 10000

# Issue Age (Integer): 20 - 59 year old
age_at_entry = rng.integers(low=20, high=60, size=point_size)

# Sex (Char)
Sex = [
    "M",
    "F"
]

sex = np.fromiter(map(lambda i: Sex[i], rng.integers(low=0, high=len(Sex), size=point_size)), np.dtype('<U1'))

# Policy Term (Integer): 10, 15, 20
policy_term = rng.integers(low=0, high=3, size=point_size) * 5 + 10


# Sum Assured (Float): 10000 - 1000000
sum_assured = np.round((1000000 - 10000) * rng.random(size=point_size) + 10000, -3)

# Issue Date:
# For existing policies, issue dates are set so that the policies are in-force on 2022/1/1.
# For new business policie,issue dates are within 3 years from 2022/1/1.

dur_offset = (policy_term + 3) * 12 * rng.random(size=point_size) - 36
mth_offset = (dur_offset // 1).astype(int)
day_offset = 1 - (dur_offset - mth_offset)
issue_mth = pd.Series([pd.Period('2022-01', 'M')] * point_size) -1 - mth_offset
day_offset = (np.ceil(issue_mth.dt.days_in_month * day_offset)).astype(int)
issue_date = (issue_mth.dt.to_timestamp().dt.to_period('D') -1 + day_offset).dt.to_timestamp().to_numpy()

# Policy Count (Integer):
policy_count = np.rint(100 * rng.random(size=point_size)).astype(int)

# Payment Frequency
payment_freq = rng.choice([1, 2, 12], size=point_size)

# Premium Payment Term
short_paidup = pd.Series(rng.choice([True, False], size=point_size))
polterm = pd.Series(policy_term)
payment_term = polterm.mask(short_paidup & (polterm==10), 5).mask(short_paidup & (polterm>=15), 10).to_numpy()

In [4]:
import pandas as pd


attrs = [
    "age_at_entry",
    "sex",
    "policy_term",
    "policy_count",
    "sum_assured",
    "issue_date",
    "payment_freq",
    "payment_term"
]

data = [
    age_at_entry,
    sex,
    policy_term,
    policy_count,
    sum_assured,
    issue_date,
    payment_freq,
    payment_term
]

model_point_table = pd.DataFrame(dict(zip(attrs, data)), index=range(1, point_size+1))
model_point_table.index.name = "policy_id"


model_point_table["uuid"] = [str(uuid.uuid1()) for i in range(0, model_point_table.shape[0])]

# shift column 'Name' to first position
first_column = model_point_table.pop('uuid')
  
# insert column using insert(position,column_name,
# first_column) function
model_point_table.insert(0, 'uuid', first_column)

model_point_table

,uuid,age_at_entry,sex,policy_term,policy_count,sum_assured,issue_date,payment_freq,payment_term
policy_id,,,,,,,,,
1,54558fe8-ce52-11ed-8c1e-eedd30f30a66,47,M,10,86,622000.0,2021-12-15,1,5
2,54559218-ce52-11ed-8c1e-eedd30f30a66,29,M,20,56,752000.0,2004-07-02,2,20
3,54559236-ce52-11ed-8c1e-eedd30f30a66,51,F,10,83,799000.0,2020-10-02,12,10
4,5455924a-ce52-11ed-8c1e-eedd30f30a66,32,F,20,72,422000.0,2011-08-05,1,10
5,5455925e-ce52-11ed-8c1e-eedd30f30a66,28,M,15,99,605000.0,2017-05-22,2,10
...,...,...,...,...,...,...,...,...,...
9996,5457cbd2-ce52-11ed-8c1e-eedd30f30a66,47,M,20,25,827000.0,2008-12-01,1,10
9997,5457cbf0-ce52-11ed-8c1e-eedd30f30a66,30,M,15,81,826000.0,2008-01-13,1,15
9998,5457cbfa-ce52-11ed-8c1e-eedd30f30a66,45,F,20,10,783000.0,2009-11-07,2,10


# [Create Premium Table](https://lifelib.io/libraries/notebooks/basiclife/create_premium_table.html)

In [6]:
%pip install modelx lifelib -q

Note: you may need to restart the kernel to use updated packages.


In [7]:
import lifelib

lifelib.create("basiclife", r"./cluster")

In [8]:
import itertools
import modelx as mx
import pandas as pd

In [ ]:
model = mx.read_model('cluster/BasicTerm_ME')
space = model.Projection

In [ ]:
space.model_point_table = model_point_table

In [ ]:
@mx.defcells
def premium_rate():
    return (1 + loading_prem()) * net_premium_pp()

In [ ]:
model_point_table['premium'] = space.premium_rate()

In [ ]:
#Add noise
for col in model_point_table.columns:
    model_point_table.loc[model_point_table.sample(frac=0.1).index, col] = pd.np.nan



In [ ]:
distorted_names = ['UUid ',
 'Age At Entry',
 'Sex ',
 'Policy Term  ',
 'Policy Count',
 'Sum Assured ',
 'Issue date',
 'Payment freq',
 'Payment_term',
 'Premium']
rename_dict = dict()
for i in range(0,len(distorted_names)):
  rename_dict.update({list(model_point_table.columns)[i]:distorted_names[i]})
model_point_table = model_point_table.rename(columns=rename_dict)
model_point_table.columns

In [ ]:
model_point_table.to_excel("model_point.xlsx")

In [ ]:
space.premium_pp.formula

In [ ]:
df = pd.DataFrame(itertools.product(range(20, 60), [10, 15, 20]), columns=['age_at_entry', 'policy_term'])
df.index += 1
df.index.name = 'point_id'
df["sum_assured"] = 1
df

In [ ]:
len(space.premium_rate())

In [ ]:
premium_rate_table = df.copy()
space.model_point_table = df
premium_rate_table["premium_rate"] = space.premium_rate()
del premium_rate_table["sum_assured"]
premium_rate_table.set_index(["age_at_entry", "policy_term"], inplace=True)
premium_rate_table["premium_rate"]   # As Series

In [ ]:
model_point_table